In [79]:
# import all necessary objects and methods for quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from random import randrange
from qiskit import qasm3
from qiskit import QuantumCircuit, transpile


In [80]:
def Send_State(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates and initializes another circuit qc2 with the same state
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qasm_string = qasm3.dumps(qc1)
    #print(qasm_string)
    #qs = qc1.qasm.split(sep=';')[4:-1]
    qs = qasm_string.split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr])
        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.h(qreg[old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        else:
            raise Exception('Unable to parse instruction')

In [81]:
no_qubits=int(input("Please Enter the no of qubits ="))

Please Enter the no of qubits = 29


In [82]:
#Step 1: Create Quantum and classical Register of n bits

qreg = QuantumRegister(no_qubits) # quantum register with 4/8/16 qubits
creg = ClassicalRegister(no_qubits) # classical register with 4/8/16 bits

#Step 2: Quantum circuit for Alice state
Alice_QC = QuantumCircuit(qreg, creg, name='Alice')


In [83]:
#Step 3: Declare 3 Array
Alice_send=[] #Initial bit string to send
Alice_basis=[] #Register to save information about encoding basis
Bob_basis=[] #Register to save information about the decoding basis

#Step 4: Creating random bit string
for i in range(no_qubits):
    bit = randrange(2)
    Alice_send.append(bit)
    
#Step 5:Preparing qubits, apply X gate if the bit is equal to 1
for i, n in enumerate(Alice_send):
    if n==1:
        Alice_QC.x(qreg[i]) # apply x-gate

#Step6: Encoding
for i in range(no_qubits):
    r=randrange(2) #Alice randomly picks a basis
    if r==0: #if the bit is 0, then she encodes in Z basis
        Alice_basis.append('Z')
    else:    #if the bit is 1, then she encodes in X basis
        Alice_QC.h(qreg[i])
        Alice_basis.append('X')

#Step 7: Quantum circuit for Bob's state
Bob_QC = QuantumCircuit(qreg, creg, name='Bob') 
Send_State(Alice_QC,Bob_QC, 'Alice') #Alice sends states to Bob

#Step 8:Bob measurements of qubits
for i in range(no_qubits):
    r=randrange(2) #Bobs randomly pick a basis
    if r==0: #if the bit is 0, then measures in Z basis
        Bob_QC.measure(qreg[i],creg[i])
        Bob_basis.append('Z')
    else:    #if the bit is 1, then measures in X basis
        Bob_QC.h(qreg[i])
        Bob_QC.measure(qreg[i],creg[i])
        Bob_basis.append('X')


In [85]:
def print_reserve(counts): # takes a dictionary variable
    for outcome in counts: # for each key-value in dictionary
        reverse_outcome = ''
        for i in outcome: # each string can be considered as a list of characters
            reverse_outcome = i + reverse_outcome # each new symbol comes before the old symbol(s)
    return reverse_outcome

In [86]:
#Introduce Simulator
simulator=AerSimulator()


# Transpile the circuit for the simulator
compiled_circuit = transpile(Bob_QC, simulator)

# Run the circuit on the simulator
job = simulator.run(compiled_circuit, shots=1)

# Get the results
result = job.result()
counts = result.get_counts(Bob_QC)
counts = print_reserve(counts)
print(counts)

10100110011100110011010010110


In [87]:
#Step 10: Saving Bob received string as a list
Bob_received = list(map(int, counts))

print("Alice sent:", Alice_send)
print("Alice encoding basis:", Alice_basis)
print("Bob received:", Bob_received)
print("Bob decoding basis:", Bob_basis,"\n")
print("Here we don't compare the Basis, hence there may be a chance of eavesdropping")
QBER=0
for i in range(0,len(Alice_send)):
    if(Alice_send[i]==Bob_received[i]):
        QBER=QBER
    else:
        QBER=QBER+1             
print("QBER_without_eavasdropping=", QBER)
QBER_percentage=QBER/len(Alice_send)*100
print("QBER_percentage_without_eavasdropping=",QBER_percentage)
QBER=0
print("\n Apply sifting operation by considering basis")

Alice sent: [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1]
Alice encoding basis: ['Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X']
Bob received: [1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
Bob decoding basis: ['X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z'] 

Here we don't compare the Basis, hence there may be a chance of eavesdropping
QBER_without_eavasdropping= 7
QBER_percentage_without_eavasdropping= 24.137931034482758

 Apply sifting operation by considering basis


In [88]:
print("Alice sent:", Alice_send)
print("Alice encoding basis:", Alice_basis)
print("Bob received:", Bob_received)
print("Bob decoding basis:", Bob_basis,"\n")

#Sifting
Alice_key=[] #Alice register for matching rounds
Bob_key=[] #Bob registers for matching rounds
for j in range(0,len(Alice_basis)): #Going through list of bases 
    if Alice_basis[j] == Bob_basis[j]: #Comparing
        Alice_key.append(Alice_send[j])
        Bob_key.append(Bob_received[j]) #Keeping key bit if bases matched
    else:
        print("Basis mismatch_"+"index =",j,",",Alice_basis[j],Bob_basis[j])
        pass #Discard round if bases mismatched
print("\n")
print("Alice_key_reduced_after_sifting =", Alice_key)
print("Bob_key_reduced_after_sifting =", Bob_key)

Alice sent: [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1]
Alice encoding basis: ['Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X']
Bob received: [1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
Bob decoding basis: ['X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z'] 

Basis mismatch_index = 0 , Z X
Basis mismatch_index = 2 , Z X
Basis mismatch_index = 6 , X Z
Basis mismatch_index = 7 , X Z
Basis mismatch_index = 8 , Z X
Basis mismatch_index = 13 , Z X
Basis mismatch_index = 14 , X Z
Basis mismatch_index = 15 , X Z
Basis mismatch_index = 16 , Z X
Basis mismatch_index = 18 , Z X
Basis mismatch_index = 20 , X Z
Basis mismatch_index = 23 , X Z
Basis mismatch_index = 25 , Z X
Basis mismatch_index = 26 , Z X
Basis mismatch_inde

In [89]:
print("Alice_key_reduced_after_sifting =", Alice_key)
print("Bob_key_reduced_after_sifting =", Bob_key)

#QBER_with_basis_matching
rounds = len(Alice_key)//3   #To divide without remainder, use //
no_of_bit_error=0
for k in range(0,rounds):
    random_bit_index = randrange(len(Alice_key))
    tested_reveal_bit_in_public_channel = Alice_key[random_bit_index]
    print ("Alice randomly selected bit index =", random_bit_index, ", and its value is = ",tested_reveal_bit_in_public_channel)
    if(Alice_key[random_bit_index]==Bob_key[random_bit_index]):
        no_of_bit_error=no_of_bit_error
    else:
        no_of_bit_error=no_of_bit_error+1
   
#removing tested bits from key strings
    del Alice_key[random_bit_index] 
    del Bob_key[random_bit_index]
print("\n")    
print("QBER=", no_of_bit_error)
QBER_percentage=no_of_bit_error/len(Alice_key)
print("QBER_percentage=",QBER_percentage)
print("Alice's secret key =", Alice_key)
print("Bob' secret key =", Bob_key)  

Alice_key_reduced_after_sifting = [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1]
Bob_key_reduced_after_sifting = [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1]
Alice randomly selected bit index = 10 , and its value is =  1
Alice randomly selected bit index = 4 , and its value is =  1
Alice randomly selected bit index = 1 , and its value is =  0
Alice randomly selected bit index = 2 , and its value is =  1


QBER= 0
QBER_percentage= 0.0
Alice's secret key = [0, 0, 1, 1, 0, 0, 1, 0, 1, 1]
Bob' secret key = [0, 0, 1, 1, 0, 0, 1, 0, 1, 1]
